In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transforms for dataset
transform = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/mvtec30/train', transform=transform)
val_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/mvtec30/val', transform=transform)
test_dataset = ImageFolder(root='/home/iai/Desktop/Jeewon/Study/Conference/Active_Learning/data/mvtec30/test', transform=transform)

/home/iai/anaconda3/envs/jwanaconda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

3201
1058
1058


In [3]:
import torch.nn as nn

# Define ResNet model
class ResNet18(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet18, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=True)
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.resnet18.conv1(x)
        x = self.resnet18.bn1(x)
        x = self.resnet18.relu(x)
        x = self.resnet18.maxpool(x)

        x = self.resnet18.layer1(x)
        x = self.resnet18.layer2(x)
        x = self.resnet18.layer3(x)
        x = self.resnet18.layer4(x)

        x = self.resnet18.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

def train(model, optimizer, criterion, labeled_loader):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    total = 0
    for images, labels in labeled_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        train_acc += (predicted == labels).sum().item()
    train_loss /= len(labeled_loader)
    train_acc /= total
    return train_loss, train_acc

In [4]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=4, verbose=False, delta=0, path='checkpoint.pt', min_epoch = 0, trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.epoch = 0
        self.min_epoch = min_epoch
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss
        self.epoch +=1
        
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            
            if self.epoch < self.min_epoch:
                self.counter = 0
                self.trace_func("Not enough epoch")
            else:
                self.counter += 1
                self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
                if self.counter >= self.patience:
                    self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [5]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import warnings
warnings.filterwarnings('ignore')

test_loss_rand = []
test_acc_rand = []
test_auc_score_rand = []
test_roc_auc_rand = []
test_precision_rand = []
test_recall_rand = []
test_f1_rand = []

num_initial_samples = 100
batch_size = 128

# Define initial labeled dataset
labeled_indices_rand = torch.randperm(len(train_dataset))[:num_initial_samples]
labeled_dataset_rand = torch.utils.data.Subset(train_dataset, labeled_indices_rand)

unlabeled_indices_rand = torch.arange(len(train_dataset))[~torch.eq(torch.arange(len(train_dataset)).unsqueeze(1), labeled_indices_rand).any(1)]
unlabeled_dataset_rand = torch.utils.data.Subset(train_dataset, unlabeled_indices_rand)

# Define data loaders
labeled_loader_rand = DataLoader(labeled_dataset_rand, batch_size=128, shuffle=True)
unlabeled_loader_rand = DataLoader(unlabeled_dataset_rand, batch_size=128, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


step = 1
# Active learning loop
while(len(unlabeled_dataset_rand) > 0):
    print("Step number:", step)

    model_rand = ResNet18(num_classes=30).to(device)
    early_stopping_rand = EarlyStopping(patience = 20, verbose = True, min_epoch = 0)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model_rand.parameters(), lr=0.001)

    for epoch in range(50):
        # Train model on labeled dataset
        train_loss, train_acc = train(model_rand, optimizer, criterion, labeled_loader_rand)
        print(f"Step : {step}, Epoch : {epoch+1} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")    

    # Evaluate model on val dataset
        model_rand.eval()
        with torch.no_grad():
            val_loss = 0.0
            val_acc = 0.0
            total = 0
            y_true = []
            y_pred = []
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model_rand(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                val_acc += (predicted == labels).sum().item()
                y_true.extend(labels.tolist())
                y_pred.extend(predicted.tolist())
            val_loss /= len(val_loader)
            val_acc /= total

            precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'macro')

            # Convert multi-class labels to binary labels (OVR)
            lb = LabelBinarizer()
            lb.fit(y_true)
            y_true_bin = lb.transform(y_true)
            y_pred_bin = lb.transform(y_pred)
            # Compute AUC score and ROC curve
            fpr, tpr, thresholds = roc_curve(y_true_bin.ravel(), y_pred_bin.ravel())
            auc_score = roc_auc_score(y_true_bin, y_pred_bin, average='macro')
            roc_auc = auc(fpr, tpr)
            
            # Early Stopping Condition
            early_stopping_rand(val_loss, model_rand)
            
            if early_stopping_rand.early_stop:
                print("Early stopping")
                print(f"Val Precision: {precision.item():.4f}, Val Recall: {recall.item():.4f}, Val F1 Score: {f1.item():.4f}")
                break
            
            print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")            
            print(f"AUC Score: {auc_score:.4f}")
            print("")
            
        model_rand.load_state_dict(torch.load('checkpoint.pt'))

    
    # Evaluate model on Test Dataset
    model_rand.load_state_dict(torch.load('checkpoint.pt'))
    model_rand.eval()
    with torch.no_grad():
        test_loss = 0.0
        test_acc = 0.0
        total = 0
        y_true = []
        y_pred = []
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_rand(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            test_acc += (predicted == labels).sum().item()
            y_true.extend(labels.tolist())
            y_pred.extend(predicted.tolist())
        test_loss /= len(test_loader)
        test_acc /= total
        
        # Convert multi-class labels to binary labels (OVR)
        lb = LabelBinarizer()
        lb.fit(y_true)
        y_true_bin = lb.transform(y_true)
        y_pred_bin = lb.transform(y_pred)
        # Compute AUC score and ROC curve
        fpr, tpr, thresholds = roc_curve(y_true_bin.ravel(), y_pred_bin.ravel())
        auc_score_rand = roc_auc_score(y_true_bin, y_pred_bin, average='macro')
        roc_auc_rand = auc(fpr, tpr)

        precision_rand, recall_rand, f1_rand, _ = precision_recall_fscore_support(y_true, y_pred, average = 'macro')
        test_loss_rand.append(test_loss)
        test_acc_rand.append(test_acc)
        test_auc_score_rand.append(auc_score_rand)
        test_roc_auc_rand.append(roc_auc_rand)
        test_precision_rand.append(precision_rand)
        test_recall_rand.append(recall_rand)
        test_f1_rand.append(f1_rand)

        
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
        print(f"AUC Score: {auc_score_rand:.4f}")
    

    # Iteration end condition
    if len(labeled_dataset_rand) > int(len(train_dataset)/2):
        break
    
    # Make model predictions on unlabeled dataset
    model_rand.eval()
    predictions = []
    with torch.no_grad():
        for images, _ in unlabeled_loader_rand:
            images = images.to(device)
            output = model_rand(images)
            predictions.append(output)
    predictions = torch.cat(predictions, dim=0)

    # Select samples to label using entropy-based method
    entropy = -torch.sum(torch.softmax(predictions, dim=1) * torch.log(torch.softmax(predictions, dim=1)), dim=1)
    _, idx = torch.topk(entropy, k = 20) # select top 20

    labeled_indices_rand = torch.cat([labeled_indices_rand, unlabeled_indices_rand[idx]])
    unlabeled_indices_rand = unlabeled_indices_rand[~torch.eq(unlabeled_indices_rand.unsqueeze(1), unlabeled_indices_rand[idx]).any(1)]

    labeled_dataset_rand = torch.utils.data.Subset(train_dataset, labeled_indices_rand)
    unlabeled_dataset_rand = torch.utils.data.Subset(train_dataset, unlabeled_indices_rand)
    print("Updated Length of labeled dataset : ",len(labeled_dataset_rand))
    print("Length of unlabeled dataset : ",len(unlabeled_dataset_rand))
    print("-----------------------------------------------------------")

    # Update labeled and unlabeled data loaders
    labeled_loader_rand = DataLoader(labeled_dataset_rand, batch_size=batch_size, shuffle=True)
    unlabeled_loader_rand = DataLoader(unlabeled_dataset_rand, batch_size=batch_size, shuffle=True)

    step+=1

Step number: 1
Step : 1, Epoch : 1 - Train Loss: 3.6251, Train Accuracy: 0.0500
Validation loss decreased (inf --> 1.359864).  Saving model ...
Val Loss: 1.3599, Val Accuracy: 0.7401
AUC Score: 0.7434

Step : 1, Epoch : 2 - Train Loss: 0.8608, Train Accuracy: 0.8200
EarlyStopping counter: 1 out of 20
Val Loss: 1.6407, Val Accuracy: 0.6654
AUC Score: 0.7435

Step : 1, Epoch : 3 - Train Loss: 0.8608, Train Accuracy: 0.8200
EarlyStopping counter: 2 out of 20
Val Loss: 1.8397, Val Accuracy: 0.6569
AUC Score: 0.7507

Step : 1, Epoch : 4 - Train Loss: 0.8608, Train Accuracy: 0.8200
EarlyStopping counter: 3 out of 20
Val Loss: 1.9357, Val Accuracy: 0.6607
AUC Score: 0.7566

Step : 1, Epoch : 5 - Train Loss: 0.8608, Train Accuracy: 0.8200
EarlyStopping counter: 4 out of 20
Val Loss: 1.9885, Val Accuracy: 0.6569
AUC Score: 0.7574

Step : 1, Epoch : 6 - Train Loss: 0.8608, Train Accuracy: 0.8200
EarlyStopping counter: 5 out of 20
Val Loss: 2.0213, Val Accuracy: 0.6474
AUC Score: 0.7543

Step : 1

In [6]:
print(test_loss_rand)
print(test_acc_rand)
print(test_auc_score_rand)
print(test_roc_auc_rand)
print(test_precision_rand)
print(test_recall_rand)
print(test_f1_rand)

[1.3827189885907702, 1.3541353245576222, 1.5248409608999889, 1.3401477350594684, 1.2127407275968127, 1.043185247315301, 1.1002838379806943, 1.4700495501359303, 1.2731655206945207, 0.7362664878859909, 0.6733606424596574, 0.7656760679350959, 0.7853845660057333, 0.6624550132287873, 0.658418756392267, 0.8095757828818427, 0.5335666756663058, 0.6061024585117897, 0.5274605837961038, 0.5392656517215073, 0.5609885338279936, 0.5985805344146987, 0.4725440206627051, 0.49848856156071025, 0.5586007891429795, 0.48105596833758885, 0.5509110457367368, 0.4693749084447821, 0.5843230585257212, 0.49956070052252877, 0.49208965928604204, 0.46322276691595715, 0.4678090372019344, 0.4830734270314376, 0.5612163866559664, 0.4777478575706482, 0.466886012090577, 0.4753226919306649, 0.45275112469163203, 0.45538345475991565, 0.6202001770337423, 0.512762564751837, 0.47659502923488617, 0.4879520965947045, 0.5079941749572754, 0.45783393043610787, 0.5149258996049563, 0.4791756057076984, 0.5535968345486455, 0.410030356711

In [7]:
test_loss_unc = []
test_acc_unc = []
test_auc_score_unc = []
test_roc_auc_unc = []
test_precision_unc = []
test_recall_unc = []
test_f1_unc = []

num_initial_samples = 100
batch_size = 128

# Define initial labeled dataset
labeled_indices_unc = torch.randperm(len(train_dataset))[:num_initial_samples]
labeled_dataset_unc = torch.utils.data.Subset(train_dataset, labeled_indices_unc)

unlabeled_indices_unc = torch.arange(len(train_dataset))[~torch.eq(torch.arange(len(train_dataset)).unsqueeze(1), labeled_indices_unc).any(1)]
unlabeled_dataset_unc = torch.utils.data.Subset(train_dataset, unlabeled_indices_unc)

# Define data loaders
labeled_loader_unc = DataLoader(labeled_dataset_unc, batch_size=128, shuffle=True)
unlabeled_loader_unc = DataLoader(unlabeled_dataset_unc, batch_size=128, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


step = 1
# Active learning loop
while(len(unlabeled_dataset_unc) > 0):
    print("Step number:", step)

    model_unc = ResNet18(num_classes=30).to(device)
    early_stopping_unc = EarlyStopping(patience = 20, verbose = True, min_epoch = 0)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model_unc.parameters(), lr=0.001)

    for epoch in range(50):
        # Train model on labeled dataset
        train_loss, train_acc = train(model_unc, optimizer, criterion, labeled_loader_unc)
        print(f"Step : {step}, Epoch : {epoch+1} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")    

    # Evaluate model on val dataset
        model_unc.eval()
        with torch.no_grad():
            val_loss = 0.0
            val_acc = 0.0
            total = 0
            y_true = []
            y_pred = []
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model_unc(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                val_acc += (predicted == labels).sum().item()
                y_true.extend(labels.tolist())
                y_pred.extend(predicted.tolist())
            val_loss /= len(val_loader)
            val_acc /= total

            precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'macro')

            # Convert multi-class labels to binary labels (OVR)
            lb = LabelBinarizer()
            lb.fit(y_true)
            y_true_bin = lb.transform(y_true)
            y_pred_bin = lb.transform(y_pred)
            # Compute AUC score and ROC curve
            fpr, tpr, thresholds = roc_curve(y_true_bin.ravel(), y_pred_bin.ravel())
            auc_score = roc_auc_score(y_true_bin, y_pred_bin, average='macro')
            roc_auc = auc(fpr, tpr)
            
            # Early Stopping Condition
            early_stopping_unc(val_loss, model_unc)
            
            if early_stopping_unc.early_stop:
                print("Early stopping")
                print(f"Val Precision: {precision.item():.4f}, Val Recall: {recall.item():.4f}, Val F1 Score: {f1.item():.4f}")
                break
            
            print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")            
            print(f"AUC Score: {auc_score:.4f}")
            print("")
            
        model_unc.load_state_dict(torch.load('checkpoint.pt'))

    
    # Evaluate model on Test Dataset
    model_unc.load_state_dict(torch.load('checkpoint.pt'))
    model_unc.eval()
    with torch.no_grad():
        test_loss = 0.0
        test_acc = 0.0
        total = 0
        y_true = []
        y_pred = []
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_unc(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            test_acc += (predicted == labels).sum().item()
            y_true.extend(labels.tolist())
            y_pred.extend(predicted.tolist())
        test_loss /= len(test_loader)
        test_acc /= total
        
        # Convert multi-class labels to binary labels (OVR)
        lb = LabelBinarizer()
        lb.fit(y_true)
        y_true_bin = lb.transform(y_true)
        y_pred_bin = lb.transform(y_pred)
        # Compute AUC score and ROC curve
        fpr, tpr, thresholds = roc_curve(y_true_bin.ravel(), y_pred_bin.ravel())
        auc_score_unc = roc_auc_score(y_true_bin, y_pred_bin, average='macro')
        roc_auc_unc = auc(fpr, tpr)

        precision_unc, recall_unc, f1_unc, _ = precision_recall_fscore_support(y_true, y_pred, average = 'macro')
        test_loss_unc.append(test_loss)
        test_acc_unc.append(test_acc)
        test_auc_score_unc.append(auc_score_unc)
        test_roc_auc_unc.append(roc_auc_unc)
        test_precision_unc.append(precision_unc)
        test_recall_unc.append(recall_unc)
        test_f1_unc.append(f1_unc)

        
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
        print(f"AUC Score: {auc_score_unc:.4f}")
    

    # Iteration end condition
    if len(labeled_dataset_unc) > int(len(train_dataset)/2):
        break
    
    # Make model predictions on unlabeled dataset
    model_unc.eval()
    predictions = []
    with torch.no_grad():
        for images, _ in unlabeled_loader_unc:
            images = images.to(device)
            output = model_unc(images)
            predictions.append(output)
    predictions = torch.cat(predictions, dim=0)

    # Select samples to label using entropy-based method
    entropy = -torch.sum(torch.softmax(predictions, dim=1) * torch.log(torch.softmax(predictions, dim=1)), dim=1)
    _, idx = torch.topk(entropy, k = 20) # select top 20

    labeled_indices_unc = torch.cat([labeled_indices_unc, unlabeled_indices_unc[idx]])
    unlabeled_indices_unc = unlabeled_indices_unc[~torch.eq(unlabeled_indices_unc.unsqueeze(1), unlabeled_indices_unc[idx]).any(1)]

    labeled_dataset_unc = torch.utils.data.Subset(train_dataset, labeled_indices_unc)
    unlabeled_dataset_unc = torch.utils.data.Subset(train_dataset, unlabeled_indices_unc)
    print("Updated Length of labeled dataset : ",len(labeled_dataset_unc))
    print("Length of unlabeled dataset : ",len(unlabeled_dataset_unc))
    print("-----------------------------------------------------------")

    # Update labeled and unlabeled data loaders
    labeled_loader_unc = DataLoader(labeled_dataset_unc, batch_size=batch_size, shuffle=True)
    unlabeled_loader_unc = DataLoader(unlabeled_dataset_unc, batch_size=batch_size, shuffle=True)

    step+=1

Step number: 1
Step : 1, Epoch : 1 - Train Loss: 3.6341, Train Accuracy: 0.0400
Validation loss decreased (inf --> 1.691642).  Saving model ...
Val Loss: 1.6916, Val Accuracy: 0.6871
AUC Score: 0.7041

Step : 1, Epoch : 2 - Train Loss: 0.8906, Train Accuracy: 0.8100
EarlyStopping counter: 1 out of 20
Val Loss: 1.7116, Val Accuracy: 0.5803
AUC Score: 0.7345

Step : 1, Epoch : 3 - Train Loss: 0.8906, Train Accuracy: 0.8100
EarlyStopping counter: 2 out of 20
Val Loss: 1.7689, Val Accuracy: 0.5652
AUC Score: 0.7410

Step : 1, Epoch : 4 - Train Loss: 0.8906, Train Accuracy: 0.8100
EarlyStopping counter: 3 out of 20
Val Loss: 1.8196, Val Accuracy: 0.5520
AUC Score: 0.7416

Step : 1, Epoch : 5 - Train Loss: 0.8906, Train Accuracy: 0.8100
EarlyStopping counter: 4 out of 20
Val Loss: 1.8565, Val Accuracy: 0.5491
AUC Score: 0.7409

Step : 1, Epoch : 6 - Train Loss: 0.8906, Train Accuracy: 0.8100
EarlyStopping counter: 5 out of 20
Val Loss: 1.8845, Val Accuracy: 0.5501
AUC Score: 0.7423

Step : 1

In [8]:
print(test_loss_unc)
print(test_acc_unc)
print(test_auc_score_unc)
print(test_roc_auc_unc)
print(test_precision_unc)
print(test_recall_unc)
print(test_f1_unc)

[1.6803721148106787, 1.0096601843833923, 0.9667922423945533, 0.9187919100125631, 1.1806141800350614, 1.1352424381507769, 0.869702685624361, 1.0246853348281648, 1.0417336606317096, 0.7711156176196204, 0.7472754187054105, 0.8016434030400382, 0.7258524199326833, 0.8048726816972097, 0.8321403364340464, 0.7852578900961412, 0.6468246794409223, 0.6571072604921129, 0.5811612630883852, 0.8093314253621631, 0.6536687893999947, 0.5975604636801614, 0.5713664872778786, 0.5364591044684252, 0.5367905249198278, 0.5686774452527364, 0.5800290173954434, 0.5925575378868315, 0.608005983961953, 0.502672928902838, 1.0062324901421864, 0.45242106169462204, 0.5159292419751486, 0.5130849530299505, 0.5949241305804915, 0.600215506222513, 0.5077625041206678, 0.576307136979368, 0.516506515443325, 0.4833182477288776, 0.6629869358407127, 0.4987487859196133, 0.5267618853184912, 0.5296222666899363, 0.42159584164619446, 0.5480133791764578, 0.47071973260285127, 0.4463435883323352, 0.46315106915103066, 0.5512856890757879, 0